In [1]:
from relation import Relation
import gensim
from gensim.models import KeyedVectors
import json
import numpy as np

In [3]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
	"""Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
	Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
		(With help from William. Thank you!)
	First, intersect the vocabularies (see `intersection_align_gensim` documentation).
	Then do the alignment on the other_embed model.
	Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
	Return other_embed.
	If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
	"""

	# make sure vocabulary and indices are aligned
	in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)

	# get the embedding matrices
	base_vecs = in_base_embed.syn0norm
	other_vecs = in_other_embed.syn0norm

	# just a matrix dot product with numpy
	m = other_vecs.T.dot(base_vecs) 
	# SVD method from numpy
	u, _, v = np.linalg.svd(m)
	# another matrix operation
	ortho = u.dot(v) 
	# Replace original array with modified one
	# i.e. multiplying the embedding matrix (syn0norm)by "ortho"
	other_embed.syn0norm = other_embed.syn0 = (other_embed.syn0norm).dot(ortho)
	return other_embed
	
def intersection_align_gensim(m1,m2, words=None):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
        The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.vocab.keys())
    vocab_m2 = set(m2.vocab.keys())

    # Find the common vocabulary
    common_vocab = vocab_m1&vocab_m2
    if words: common_vocab&=set(words)

    # If no alignment necessary because vocab is identical...
    if not vocab_m1-common_vocab and not vocab_m2-common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.vocab[w].count + m2.vocab[w].count,reverse=True)

    # Then for each model...
    for m in [m1,m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.vocab[w].index for w in common_vocab]
        old_arr = m.syn0norm
        new_arr = np.array([old_arr[index] for index in indices])
        m.syn0norm = m.syn0 = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        m.index2word = common_vocab
        old_vocab = m.vocab
        new_vocab = {}
        for new_index,word in enumerate(common_vocab):
            old_vocab_obj=old_vocab[word]
            new_vocab[word] = gensim.models.word2vec.Vocab(index=new_index, count=old_vocab_obj.count)
        m.vocab = new_vocab

    return (m1,m2)

In [106]:
embeddings_t0 = KeyedVectors.load('../code/embeddings/year1950_1954')
embeddings_t1 = KeyedVectors.load('../code/embeddings/year1955_1959')

2018-11-20 15:11:06,170 : INFO : loading Word2VecKeyedVectors object from ../code/embeddings/year1950_1954
2018-11-20 15:11:07,450 : INFO : loading vectors from ../code/embeddings/year1950_1954.vectors.npy with mmap=None
2018-11-20 15:11:07,620 : INFO : setting ignored attribute vectors_norm to None
2018-11-20 15:11:07,621 : INFO : loaded ../code/embeddings/year1950_1954
2018-11-20 15:11:07,622 : INFO : loading Word2VecKeyedVectors object from ../code/embeddings/year1955_1959
2018-11-20 15:11:10,917 : INFO : loading vectors from ../code/embeddings/year1955_1959.vectors.npy with mmap=None
2018-11-20 15:11:11,112 : INFO : setting ignored attribute vectors_norm to None
2018-11-20 15:11:11,112 : INFO : loaded ../code/embeddings/year1955_1959


In [107]:
embeddings_t1 = smart_procrustes_align_gensim(embeddings_t0, embeddings_t1)

/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).


TypeError: 'NoneType' object is not subscriptable

In [81]:
model.wv.most_similar(positive=['vrouw', 'koning'], negative=['man'])

/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


KeyError: "word 'man' not in vocabulary"

In [88]:
t1.wv.doesnt_match("ontbijt lunch maaltijd koek".split())

/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'koek'

In [90]:
t0['oorlog']

0.00969977

In [48]:
cats = json.load(open("../data/semtest.json"))

In [49]:
Relation.create_set(cats, "../data/question-words.txt")

In [50]:
rel = Relation("../data/question-words.txt")
scores = rel.test_model(model)

/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [56]:
scores[1].keys()

dict_keys(['section', 'correct', 'incorrect'])

In [64]:
{k: v for k, v in scores[0].items() if k.startswith('incorrect')}

{'incorrect': [('DANS', 'DANSEN', 'LOOP', 'LOPEN'),
  ('DANS', 'DANSEN', 'SCHIET', 'SCHIETEN'),
  ('DANS', 'DANSEN', 'PRAAT', 'PRATEN'),
  ('DANS', 'DANSEN', 'DENK', 'DENKEN'),
  ('DANS', 'DANSEN', 'GA', 'GAAN'),
  ('DANS', 'DANSEN', 'BESCHRIJF', 'BESCHRIJVEN'),
  ('DANS', 'DANSEN', 'ZWEM', 'ZWEMMEN'),
  ('DANS', 'DANSEN', 'VLIEG', 'VLIEGEN'),
  ('DANS', 'DANSEN', 'KIJK', 'KIJKEN'),
  ('DANS', 'DANSEN', 'ZING', 'ZINGEN'),
  ('DANS', 'DANSEN', 'ZEG', 'ZEGGEN'),
  ('DANS', 'DANSEN', 'SCHREEUW', 'SCHREEUWEN'),
  ('DANS', 'DANSEN', 'REN', 'RENNEN'),
  ('DANS', 'DANSEN', 'LEES', 'LEZEN'),
  ('DANS', 'DANSEN', 'RACE', 'RACEN'),
  ('DANS', 'DANSEN', 'KEN', 'KENNEN'),
  ('DANS', 'DANSEN', 'BEREKEN', 'BEREKENEN'),
  ('DANS', 'DANSEN', 'SPEEL', 'SPELEN'),
  ('DANS', 'DANSEN', 'VERVEEL', 'VERVELEN'),
  ('DANS', 'DANSEN', 'ZIE', 'ZIEN'),
  ('DANS', 'DANSEN', 'SCHRIJF', 'SCHRIJVEN'),
  ('DANS', 'DANSEN', 'LOOP', 'LOPEN'),
  ('DANS', 'DANSEN', 'LEG', 'LEGGEN'),
  ('DANS', 'DANSEN', 'ONDERHANDEL', 'O

In [77]:
model.most_similar('verenigde_staten')

/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('amerika', 0.7645782828330994),
 ('amerikaanse', 0.6737560033798218),
 ('economische_hulp', 0.6551260948181152),
 ('militaire_hulp', 0.6432616710662842),
 ('amerikaanse_regering', 0.6407544612884521),
 ('wapenleveranties', 0.6273886561393738),
 ('washingtonverenigde', 0.6253843307495117),
 ('amerikaanse_congres', 0.6228297352790833),
 ('amerikaanse_hulp', 0.6185386776924133),
 ('washingtonreuter', 0.6123563051223755)]